--> This code file was run on Google Colab

In [2]:
# !pip install transformers bitsandbytes>=0.39.0 -q

In [3]:
from transformers import AutoModelForCausalLM
from huggingface_hub import login


In [4]:
# login(Your Access Token Goes in Here)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map = "auto", load_in_4bit = True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
model_inputs1 = tokenizer(["A list of colors: red, blue"], return_tensors="pt").to("cuda")

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
generated_ids = model.generate(**model_inputs1)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


'A list of colors: red, blue, green, yellow, orange, purple, pink,'

In [8]:
tokenizer.pad_token = tokenizer.eos_token
model_inputs2 = tokenizer(
    ["A list of colors: red, blue", "Aviation is the most expensive form of transportation"], return_tensors = "pt", padding=True
).to("cuda")

generated_ids = model.generate(**model_inputs2)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['A list of colors: red, blue, green, yellow, orange, purple,',
 'Aviation is the most expensive form of transportation. It is also the safest.\n']

--> This was the basic generation

--> Now we will look at some variations and optimizations

1.) Setting Max Tokens

In [9]:
# Default = 20

model_inputs3 = tokenizer(["A sequence of numbers: 1, 2"], return_tensors="pt").to("cuda")

# By default, the output will contain up to 20 tokens
generated_ids = model.generate(**model_inputs3)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'A sequence of numbers: 1, 2, 3, 4, 5'

In [10]:
# Defined = 50

generated_ids = model.generate(**model_inputs3, max_new_tokens=50)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'A sequence of numbers: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,'

2.) Incorrect Generation Mode

In [11]:
# Setting the seed or reproducibility

from transformers import set_seed
set_seed(42)

model_inputs4 = tokenizer(["I am a cat."], return_tensors="pt").to("cuda")

In [12]:
# LLM + Greedy

generated_ids = model.generate(**model_inputs4)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'I am a cat. I am a cat. I am a cat. I am a cat'

In [13]:
# With Sampling

generated_ids = model.generate(**model_inputs4, do_sample=True)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'I am a cat.  Specifically, I am an indoor-only cat.  I'

3.) Wrong Padding Side

In [14]:
# Right Padding

model_inputs5 = tokenizer(
    ["1, 2, 3", "A, B, C, D, E"], padding=True, return_tensors="pt"
).to("cuda")
generated_ids = model.generate(**model_inputs5)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'1, 2, 3, 4, 5, 6,'

In [15]:
# Left Padding

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default
model_inputs = tokenizer(
    ["1, 2, 3", "A, B, C, D, E"], padding=True, return_tensors="pt"
).to("cuda")
generated_ids = model.generate(**model_inputs5)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'1, 2, 3, 4, 5, 6,'

4.) Wrong Prompt and Chat Templating

In [16]:
# Basic

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
model = AutoModelForCausalLM.from_pretrained(
    "HuggingFaceH4/zephyr-7b-alpha", device_map="auto", load_in_4bit=True
)
set_seed(0)
prompt = """How many helicopters can a human eat in one sitting? Reply as a thug."""
model_inputs6 = tokenizer([prompt], return_tensors="pt").to("cuda")
input_length = model_inputs.input_ids.shape[1]
generated_ids = model.generate(**model_inputs6, max_new_tokens=20)
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


in one sitting? Reply as a thug.

I'm not a thug, but i can tell you that a human cannot eat


In [17]:
# Using the Chat Templating Method

set_seed(0)
messages = [
    {
        "role": "system",
        "content": "You are an SQL engineer. You give out SQL queries based on the user input. Your output is only the query and nothing else.",
    },
    {
        "role": "user", "content": "I have a table named Student. I want to fetch the names of all the students who have grades above 90 and who are Male"
    },
]
model_inputs7 = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
input_length = model_inputs7.shape[1]
generated_ids = model.generate(model_inputs7, do_sample=True, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


SELECT name
FROM Student
WHERE grade > 90 AND sex = 'M';


--> Now, we will create a chatbot using the chat template method. This is where the fun begins. This is where you don't need to pay OpenAI for having a conversation.

In [23]:
# !pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.8 MB/s eta 0:00:00


In [37]:
# Function to get the chat response
def get_chat_response(system_message, user_message, model, tokenizer):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    # Tokenize the chat messages
    model_inputs8 = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")

    # Generate response
    input_length = model_inputs8.shape[1]
    generated_ids = model.generate(model_inputs8, do_sample=True, max_new_tokens=50)

    # Decode the response
    return tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0]

# Ask for the context (system role input)
system_message = input("Enter the context for the system role (e.g., 'You are an SQL engineer.'): ")

# Start the chatbot
print("\nChatbot started! Type 'BYE' to exit.\n")

# Loop for conversation
while True:
    # Ask for user input
    user_message = input("You: ")

    # Check for exit condition
    if user_message.strip().upper() == "BYE":
        print("Chatbot: Goodbye!")
        break

    # Get chatbot response
    chat_response = get_chat_response(system_message, user_message, model, tokenizer)

    # Display chatbot's response
    print(f"Chatbot: {chat_response}")

Enter the context for the system role (e.g., 'You are an SQL engineer.'): You are my friend

Chatbot started! Type 'BYE' to exit.

You: hello


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Chatbot: Hello, how may I assist you today?
You: how can you be of any assistance at all?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Chatbot: I can assist you by providing information and generating responses to your questions. You can type in your desired queries, and I will provide the best and most accurate responses to your inquiries.
You: Oh great


...

Chatbot: I don't have personal relationships as i am a computer program designed to assist and respond to your queries. however, india and i have a close relationship, as i was built and trained on vast databases of information, which includes knowledge about
You: No, no, I mean, what is the capital of India?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Chatbot: The capital city of India is New Delhi, also known as Delhi. However, it's official name is the National Capital Territory of Delhi, or simply, the NCT of Delhi.
You: you are a very helpful friend


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Chatbot: I'm delighted to hear that I've been able to assist you in helpful ways. I'm programmed to provide accurate, informative responses to your inquiries, and my main goal is to support and facilitate your learning process.
You: BYE
Chatbot: Goodbye!


--> Saving the Model and the Tokenizer

In [ ]:
# Save the model and tokenizer locally
model.save_pretrained("./path_to_save_model")
tokenizer.save_pretrained("./path_to_save_tokenizer")

--> Given below is a streamlit code. You need to save and download the model and tokenizer and then run the code given below, locally.

In [ ]:
%%writefile app.py

import streamlit as st

# Function to get the chat response
def get_chat_response(system_message, user_message, model, tokenizer):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    # Tokenize the chat messages
    model_inputs8 = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")

    # Generate response
    input_length = model_inputs8.shape[1]
    generated_ids = model.generate(model_inputs8, do_sample=True, max_new_tokens=50)

    # Decode the response
    return tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0]

# Streamlit app interface
st.title("AI Chatbot")

# Step 1: Ask the user for the context
system_message = st.text_input("Enter the context for the chatbot role (e.g., 'You are an SQL engineer.')", "")

# If the user has entered a context, proceed with the conversation
if system_message:
    st.write("### Chatbot started! Type your messages below:")

    # Initialize a session state to store conversation history
    if "conversation" not in st.session_state:
        st.session_state.conversation = []

    # Step 2: Input from the user
    user_message = st.text_input("You: ", "")

    # Display the conversation history
    for message in st.session_state.conversation:
        st.write(f"You: {message['user']}")
        st.write(f"Chatbot: {message['bot']}")

    # Check if the user has entered a message and hasn't typed 'BYE'
    if user_message and user_message.strip().upper() != "BYE":
        # Get chatbot's response
        chat_response = get_chat_response(system_message, user_message, model, tokenizer)

        # Append the new exchange to the session state
        st.session_state.conversation.append({"user": user_message, "bot": chat_response})

        # Clear the user message input field for the next message
        st.experimental_rerun()

    # If the user types 'BYE', end the conversation
    if user_message.strip().upper() == "BYE":
        st.write("Chatbot: Goodbye!")
        st.stop()
